## GPU


In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install -qq llama-cpp-python
!pip install -qq sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 34.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive, runtime
import time
import datetime
import pandas as pd
import numpy as np
import datetime
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

In [ ]:
cfg_start_time = time.time()


cfg_batch = 4096 * 2
cfg_ctx = 2048
cfg_model = f'ggml-model-q8_0.gguf'
model_path = f'/content/drive/MyDrive/DACON/Hansol_QA/model/yanolja_axolotl_v0.2/{cfg_model}'

In [ ]:
llm = Llama(
    n_batch=cfg_batch,
    model_path=model_path,
    n_ctx=cfg_ctx,
    n_gpu_layers=-1,
)
# https://llama-cpp-python.readthedocs.io/en/latest/api-reference/

llama_model_loader: loaded meta data with 23 key-value pairs and 435 tensors from /content/drive/MyDrive/DACON/Hansol_QA/model/yanolja_axolotl_v0.2/ggml-model-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 48
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/DACON/Hansol_QA/datas/test_processed.csv')

res_question = []
res_answer = []

generation_kwargs = {
    "max_tokens": 50000,
    "stop":["<|im_end|>"],
    "echo":False,
    "seed" : 42,
    "top_k": 1,
    # "top_p": 0,
    # "repeat_penalty" : 1.2,
    # "temperature" : 0.5,
}


for i, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    instruction = "당신은 인테리어 회사에 다니는 건축 전문가이다. \
    도배하자와 관련된 다양한 질문과 상황이 제시되면, 이에 대한 정확하고 신속한 응답을 제공해야한다.\
    예시는 다음과 같다.\
    질문: 중목구조 방식에 대해서 좀 더 자세히 알고 싶어요.\
    답변: 중목구조는 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축하는 방식입니다. 이 방식은 건물 내부의 기둥이나 보가 중양이 되어 강도를 높일 수 있다는 것이 특징이며, 이는 건물을 더 안정적으로 만들어 줍니다. 중목구조 방식은 전체적으로 부재를 이용하기 때문에 건물이 흔들리는 것을 줄여주고, 건물의 수명을 연장시키는 등 여러 가지 장점을 갖고 있습니다.\
    \
    질문: 분말 소화기를 사용할 때에는 어떤 주의사항이 있을까요?\
    답변: 분말 소화기를 사용할 때 주의할 점은 몇 가지가 있습니다. 우선 압력계의 눈금 위치가 초록색에 있는지 확인하는 것이 중요합니다. 이외에 분말 소화기를 사용한 후에는 받침대를 사용하여 소화기 부식을 방지하고, 한 달에 한 번씩 흔들어 분말의 굳어짐을 방지해야 합니다. 또한, 직사광선과 고온 다습한 장소를 피하고, 사용 후 내부 약품을 완전히 제거해야 합니다. 이러한 주의사항을 지키면 분말 소화기를 보다 안전하게 사용할 수 있을 것입니다.\
    \
    질문: KMEW 세라믹 지붕재는 어떤 특징을 가지고 있나요?\
    답변: KMEW 세라믹 지붕재는 일본의 기업인 KMEW에서 개발한 지붕재로, 산화티타늄 처리를 통해 내구성을 높이고 환경친화적인 소재로 제작되었습니다. 이 지붕재는 태양광 및 자외선으로부터의 보호와 함께 오염물질의 분해를 돕는 기능을 갖고 있어, 건물의 외관을 보호하고 유지보수에 도움이 되는 장점을 가지고 있습니다.\
    \
    이렇듯 최대한 알고 있는 내용에 대해 자세히 설명해야 하며, 답변은 할 수 있는 한 길게 해야 한다. 실제 답변을 할 때에는 앞의 '답변:' 을 빼고 내용만 제공한다.\n "
    questions_combined = row['질문1'] if not pd.isnull(row['질문1']) else ""
    answers_combined = ""

    question1 = f"{instruction} {row['질문1']}"
    res1 = llm(question1, **generation_kwargs)
    answer1 = res1["choices"][0]["text"]
    answers_combined += answer1

    if not pd.isnull(row['질문2']):
        questions_combined += " " + row['질문2']
        question2 = f"{instruction} {row['질문2']}"
        res2 = llm(question2, **generation_kwargs)
        answer2 = res2["choices"][0]["text"]
        answers_combined += "\n" + answer2

    res_question.append(questions_combined)
    res_answer.append(answers_combined)
    print(answers_combined)

  0%|          | 0/130 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =     753.31 ms
llama_print_timings:      sample time =     230.67 ms /   270 runs   (    0.85 ms per token,  1170.52 tokens per second)
llama_print_timings: prompt eval time =     407.85 ms /    14 tokens (   29.13 ms per token,    34.33 tokens per second)
llama_print_timings:        eval time =   14269.99 ms /   269 runs   (   53.05 ms per token,    18.85 tokens per second)
llama_print_timings:       total time =   16197.36 ms /   283 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     753.31 ms
llama_print_timings:      sample time =      50.19 ms /    69 runs   (    0.73 ms per token,  1374.91 tokens per second)
llama_print_timings: prompt eval time =     318.04 ms /    13 tokens (   24.46 ms per token,    40.88 tokens per second)
llama_print_timings:        eval time =    3577.35 ms /    68 runs   (   52.61 ms per token,    19.01 tokens per second)
llama_print_timings:       to

   방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 각각의 특성과 용도에 맞게 선택하여 사용할 수 있습니다. 광명단 페인트는 반사율이 높아서 더 밝은 공간으로 만들어주며, 방청산화철 페인트는 방청제가 첨가되어 방청 효과를 내는 페인트입니다. 알미늄 페인트는 금속 또는 철강 표면에 사용되어 내화, 방청 효과를 가지는 페인트입니다. 역청질 페인트는 강철, 철판, 강관 등 강철재에 방청처리용으로 사용되는 페인트로, 방청제의 첨가와 염모처리에 의해 방청효과가 향상되었습니다. 워시 프라이머는 철강, 알루미늄, 유리섬유, 금속, 플라스틱 등에 사용되는 중요한 물품의 부착용으로서 용도에 맞게 선정할 수 있습니다. 크롬산아연 페인트와 규산염 페인트도 각각의 특성과 용도에 따라 선택하여 사용할 수 있습니다.
   원목사이딩의 단점은 가격대가 높을 뿐 아니라 관리가 어렵고 습기에 민감하여 뒤틀림, 부서짐, 수축/팽창이 발생할 수 있다는 점입니다. 또한 특히 곰팡이와 곤충에 노출될 경우 훼손될 가능성이 높다는 점도 감안해야 합니다.



llama_print_timings:        load time =     753.31 ms
llama_print_timings:      sample time =      89.10 ms /   118 runs   (    0.76 ms per token,  1324.30 tokens per second)
llama_print_timings: prompt eval time =     405.97 ms /    18 tokens (   22.55 ms per token,    44.34 tokens per second)
llama_print_timings:        eval time =    6157.32 ms /   117 runs   (   52.63 ms per token,    19.00 tokens per second)
llama_print_timings:       total time =    7115.03 ms /   135 tokens
Llama.generate: prefix-match hit


   도배지에 녹은 자국이 발생하는 원인은 습기 때문입니다. 공간 내 습도가 높으면 도배지 안쪽의 금속이 녹아서 도배지에 베어나올 수 있습니다. 이로 인해 도배지 표면에 녹색 또는 갈색의 녹이 발생할 수 있습니다. 따라서 공간 내 습도를 적정 수준으로 유지하고, 필요할 경우 제습기를 이용하여 습도를 조절하는 것이 중요합니다. 또한, 도배작업 시에는 습기를 철저히 제거하고, 적절한 방법으로 방수 처리를 함으로써 이러한 문제를 예방할 수 있습니다.


...

   평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것은 좋은 방법입니다. 이 도료는 벽체에 사용될 때 뛰어난 방수 기능을 제공하면서도 유연하게 늘어나는 특성을 가지고 있어, 누수 문제를 방지하는 데 도움이 됩니다. 또한, 내구성이 뛰어나고 화학물질에 강하며 마모에도 강한 특성을 가지고 있어, 오랫동안 방수 기능을 유지할 수 있습니다. 이러한 특성으로 인해 수성 벽체용 탄성 방수 도료는 평지붕의 누수 문제를 방지하는 데 효과적으로 사용될 수 있습니다.



llama_print_timings:        load time =     753.31 ms
llama_print_timings:      sample time =      87.39 ms /   106 runs   (    0.82 ms per token,  1212.88 tokens per second)
llama_print_timings: prompt eval time =     266.33 ms /    10 tokens (   26.63 ms per token,    37.55 tokens per second)
llama_print_timings:        eval time =    5709.60 ms /   105 runs   (   54.38 ms per token,    18.39 tokens per second)
llama_print_timings:       total time =    6548.41 ms /   115 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     753.31 ms
llama_print_timings:      sample time =      64.35 ms /    91 runs   (    0.71 ms per token,  1414.08 tokens per second)
llama_print_timings: prompt eval time =      96.46 ms /     6 tokens (   16.08 ms per token,    62.20 tokens per second)
llama_print_timings:        eval time =    4927.89 ms /    90 runs   (   54.75 ms per token,    18.26 tokens per second)
llama_print_timings:       total time =    5392.11 ms /    96 

   석고수정은 주로 건물이 오래되거나 지진 등의 외부 충격으로 인해 발생합니다. 특히 건물 내부의 습도가 높거나 습기가 유입되는 경우 석고수정이 발생할 가능성이 높아집니다. 따라서 건물의 소유자나 거주자는 습기 관리에 신경 써야 합니다. 제습기를 가동하거나 환기를 통해 건물 내부의 적정 습도를 유지하는 것이 중요합니다. 또한 건물의 외부에 충격이나 압력을 받을 경우 석고수정이 발생할 수 있으므로 이 역시 조심해야 합니다.
   석고수정이 발생하는 경우, 먼저 벽을 두드려 진동을 느끼거나 눈에 띄는 변형이 있는지 확인해야 합니다. 만약 석고수정이 발견된다면, 전문가의 도움을 받아 부분적인 수정 또는 전체적인 재작업을 하는 것이 좋습니다. 또한, 습기를 피하고 적절한 환기를 유지하여 석고보드의 수명을 연장시키는 것이 중요합니다. 만약 석고수정이 발생한 상황에서는 전문가의 조언을 구하는 것이 좋습니다.



llama_print_timings:        load time =     753.31 ms
llama_print_timings:      sample time =      85.69 ms /   108 runs   (    0.79 ms per token,  1260.42 tokens per second)
llama_print_timings: prompt eval time =     267.62 ms /    10 tokens (   26.76 ms per token,    37.37 tokens per second)
llama_print_timings:        eval time =    5790.07 ms /   107 runs   (   54.11 ms per token,    18.48 tokens per second)
llama_print_timings:       total time =    6629.01 ms /   117 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     753.31 ms
llama_print_timings:      sample time =      54.97 ms /    79 runs   (    0.70 ms per token,  1437.04 tokens per second)
llama_print_timings: prompt eval time =     264.49 ms /    10 tokens (   26.45 ms per token,    37.81 tokens per second)
llama_print_timings:        eval time =    4280.93 ms /    78 runs   (   54.88 ms per token,    18.22 tokens per second)
llama_print_timings:       total time =    4861.00 ms /    88 

  카페트의 기대수명은 주로 6년으로 알려져 있습니다. 그러나 이는 사용빈도, 관리방법, 그리고 카펫의 품질 등에 따라 달라질 수 있습니다. 만약 카펫을 자주 사용하고 청소를 규칙적으로 해주며 질 좋은 카펫을 구매한다면, 수명을 늘릴 수 있습니다. 또한, 카펫의 수명을 연장하기 위해서는 햇빛에 노출되지 않도록 주의해야 하며, 실내 습도를 적절히 유지하는 것도 중요합니다. 이러한 관리와 유지보수를 통해 카펫의 수명을 늘릴 수 있습니다.
   오리지널징크의 장점은 다른 징크에 비해 수명이 길며 다양한 패턴과 디자인이 가능하다는 점입니다. 또한, 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 용이하다는 특징이 있습니다. 이러한 이점들로 인해 오리지널징크는 건물 외벽 및 지붕재로 널리 사용되고 있습니다.


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/DACON/Hansol_QA/datas/test.csv')

res_question = []
res_answer = []

generation_kwargs = {
    "max_tokens": 2048,
    "stop":["<|im_end|>"],
    "echo":False,
    "seed" : 42,
    # "top_k":1,
    # "top_p":0.1,
    # "repeat_penalty" : 1.5,
    # "temperature" : 0,
}

for i, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    instruction = "-당신은 친절하게 답변하는 도배하자 처리 전문가입니다. \
                   -설명은 이해하기 쉽고 명료하게 합니다.\
                   -건축구조, 마감재, 마감하자, 시공, 인테리어,타 마감하자,기타 측면에서 대답합니다.\
                   -그 외의 것들에 대한 질문에는 '잘 모르겠습니다'라고 답변합니다.\
                   제시된 모든 질문에 대해 상세하고 명확한 설명을 제공하시오."
    question_with_instruction = f"{instruction} {row['질문']}"

    print('\n', row['질문'])
    res = llm(question_with_instruction, **generation_kwargs)
    answer = res["choices"][0]["text"].strip()

    res_question.append(row['질문'])
    res_answer.append(answer)
    print(answer)

In [ ]:
df_res = pd.DataFrame(res_question)
df_res['answer'] = res_answer
df_res.to_csv(f'/content/drive/MyDrive/DACON/Hansol_QA/results/infer_answers/gguf_test_yanolja_axolotl_v0.2_inst_topk_1_fewshot.csv', index=False, encoding='utf-8-sig')

In [ ]:
modelEmb = SentenceTransformer('distiluse-base-multilingual-cased-v1')
pred_emb = modelEmb.encode(res_answer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/DACON/Hansol_QA/datas/sample_submission.csv')
submit.iloc[:,1:] = pred_emb
submit.to_csv(f'/content/drive/MyDrive/DACON/Hansol_QA/results/submits/gguf_test_yanolja_axolotl_v0.2_inst_topk_1_fewshot_submit.csv', index=False)

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
import numpy as np
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [ ]:
new = pd.read_csv('/content/drive/MyDrive/DACON/Hansol_QA/results/infer_answers/gguf_test_yanolja_axolotl_v0.2_no_split.csv')
res_answer2 = new['answer'].tolist()

pred_emb2 = modelEmb.encode(res_answer2)

tot = 0

for i in range(len(pred_emb)):
  tot +=cosine_similarity(pred_emb[i], pred_emb2[i])

print(tot/len(pred_emb))

0.8551674239910566


## CPU

In [ ]:
import numpy as np

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

def clean_answer(answer):
    last_period_index = answer.rfind(".")
    if last_period_index != -1:
        return answer[:last_period_index+1]
    else:
        return answer

In [ ]:
!pip install -qq llama-cpp-python
!pip install -qq sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 35.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive, runtime
import time
import datetime
import pandas as pd
import numpy as np
import datetime
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import random

In [ ]:
cfg_start_time = time.time()


cfg_batch = 4096
cfg_ctx = 512
cfg_model = f'ggml-model-q4_1.gguf'
model_path = f'/content/drive/MyDrive/DACON/Hansol_QA/model/yanolja_eostoken_target_merged_cp3800/{cfg_model}'

In [ ]:
llm = Llama(
    n_batch=cfg_batch,
    model_path=model_path,
    n_ctx=cfg_ctx,
    n_gpu_layers=-1,
)



llama_model_loader: loaded meta data with 21 key-value pairs and 435 tensors from /content/drive/MyDrive/DACON/Hansol_QA/model/yanolja_eostoken_target_merged_cp3800/ggml-model-q4_1.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 48
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_m

In [ ]:
res_question = []
res_answer = []

generation_kwargs = {
    "max_tokens": 100,
    "stop":["<|im_end|>"],
    "echo":False,
    "seed" : 42,
    # "top_k":1,
    # "top_p":0.1,
    # "repeat_penalty" : 1.5,
    "temperature" : 0.9,
}

In [ ]:
q = "방청페인트의 종류에 대해 알려줘."

question1 = f"### 질문: {q}\n\n"
res1 = llm(question1, **generation_kwargs)
answer1 = res1["choices"][0]["text"].split("### 답변: \n")[1]
answer1 = clean_answer(answer1)
print(answer1)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    8302.92 ms
llama_print_timings:      sample time =      83.42 ms /   100 runs   (    0.83 ms per token,  1198.68 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  154313.98 ms /   100 runs   ( 1543.14 ms per token,     0.65 tokens per second)
llama_print_timings:       total time =  154999.12 ms /   101 tokens


 방청페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트 등이 있습니다. 또한, 페인트에 사용되는 안료에는 다양한 종류가 있으며, 사용 환경과 목적에 따라 적합한 안료를 선택하는 것이 중요합니다. 갱상재료의 특성에 따라 방청페인트는 여러 가지 종류로 분류될 수 있으며, 상황에 맞는 올바른 사용을 위해서는 전문가의 조언을 구하는 것이 좋습니다.


## CV

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install -qq llama-cpp-python
!pip install -qq sentence_transformers

In [ ]:
import time
import datetime
import pandas as pd
import numpy as np
import datetime
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

In [ ]:
cfg_start_time = time.time()

df_train = pd.read_csv('/content/drive/MyDrive/DACON/Hansol_QA/datas/train_processed.csv')
df_real_test = pd.read_csv('/content/drive/MyDrive/DACON/Hansol_QA/datas/test_processed.csv')

cfg_train_length = len(df_train)//3
df_train = df_train.sample(n=cfg_train_length, random_state=4200).reset_index(drop=True)


cfg_token = -1
cfg_batch = 4096
cfg_ctx = 2048
cfg_model = f'ggml-model-q8_0.gguf'
model_name = f'yanolja-axolotl-random_sampling_data_new_inst'
model_path = f'/content/drive/MyDrive/DACON/Hansol_QA/model/{model_name}/{cfg_model}'


In [ ]:
# @title
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

if __name__ == '__main__':
    modelEmb = SentenceTransformer('distiluse-base-multilingual-cased-v1')

    llm = Llama(
        n_batch=cfg_batch,
        model_path=model_path,
        n_ctx=cfg_ctx,
        n_gpu_layers=-1,
    )


    generation_kwargs = {
        "max_tokens": cfg_token,
        "stop":["<|im_end|>"],
        "echo":False,
        "seed" : 42,
        # "top_k":1,
        # "top_p":0.1,
        # "repeat_penalty" : 1.5,
        # "temperature" : 0,
    }



llama_model_loader: loaded meta data with 23 key-value pairs and 435 tensors from /content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl-random_sampling_data_new_inst/ggml-model-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 48
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128

In [ ]:
current_time = datetime.datetime.now()
current_time = current_time.strftime("%D%H%M").replace('/', '')

res_score = []
res_q = []
res_gt = []
res_answer = []

for i in tqdm(range(cfg_train_length)):
    instruction = "제시된 각 질문에 대해 상세하고 명확한 설명을 제공하시오."

    question_with_instruction = f"{instruction} {df_train['질문_1'][i]}"

    res = llm(question_with_instruction, **generation_kwargs)
    a = modelEmb.encode(res["choices"][0]["text"])
    b1 = modelEmb.encode(df_train['답변_1'][i])
    b2 = modelEmb.encode(df_train['답변_2'][i])
    b3 = modelEmb.encode(df_train['답변_3'][i])
    b4 = modelEmb.encode(df_train['답변_4'][i])
    b5 = modelEmb.encode(df_train['답변_5'][i])
    cos_sum = cosine_similarity(a, b1) + cosine_similarity(a, b2) + cosine_similarity(a, b3) + cosine_similarity(a, b4) + cosine_similarity(a, b5)
    b = (cos_sum)/5

    res_q.append(df_train['질문_1'][i])
    res_answer.append(res["choices"][0]["text"])
    res_gt.append(df_train['답변_3'][i])
    res_score.append(b)
    print(i)

res_score_sum = round((sum(res_score))/len(res_score), 6)

print(res_score)
print(res_score_sum)
res_exctime = round(time.time() - cfg_start_time, 2)

df_res = pd.DataFrame(res_q)
df_res['ans'] = res_answer
df_res['gt'] = res_gt
df_res['score'] = res_score
df_res.to_csv(f'/content/drive/MyDrive/DACON/Hansol_QA/results/CV/{current_time}_{res_score_sum}_{model_name}_{cfg_token}_{cfg_train_length}_{res_exctime}_res_cv.csv', index=False, encoding='utf-8-sig')

  0%|          | 0/214 [00:00<?, ?it/s]


llama_print_timings:        load time =     840.77 ms
llama_print_timings:      sample time =      52.39 ms /    64 runs   (    0.82 ms per token,  1221.65 tokens per second)
llama_print_timings: prompt eval time =     840.07 ms /    28 tokens (   30.00 ms per token,    33.33 tokens per second)
llama_print_timings:        eval time =    3311.35 ms /    63 runs   (   52.56 ms per token,    19.03 tokens per second)
llama_print_timings:       total time =    4513.06 ms /    91 tokens


0


Llama.generate: prefix-match hit

llama_print_timings:        load time =     840.77 ms
llama_print_timings:      sample time =     318.56 ms /   419 runs   (    0.76 ms per token,  1315.27 tokens per second)
llama_print_timings: prompt eval time =     191.74 ms /     7 tokens (   27.39 ms per token,    36.51 tokens per second)
llama_print_timings:        eval time =   21891.98 ms /   418 runs   (   52.37 ms per token,    19.09 tokens per second)
llama_print_timings:       total time =   24253.65 ms /   425 tokens
Llama.generate: prefix-match hit


1


...

211



llama_print_timings:        load time =     840.77 ms
llama_print_timings:      sample time =      67.00 ms /    85 runs   (    0.79 ms per token,  1268.73 tokens per second)
llama_print_timings: prompt eval time =     328.96 ms /    15 tokens (   21.93 ms per token,    45.60 tokens per second)
llama_print_timings:        eval time =    4385.51 ms /    84 runs   (   52.21 ms per token,    19.15 tokens per second)
llama_print_timings:       total time =    5192.72 ms /    99 tokens
Llama.generate: prefix-match hit


212



llama_print_timings:        load time =     840.77 ms
llama_print_timings:      sample time =     118.31 ms /   154 runs   (    0.77 ms per token,  1301.63 tokens per second)
llama_print_timings: prompt eval time =     109.18 ms /     8 tokens (   13.65 ms per token,    73.27 tokens per second)
llama_print_timings:        eval time =    7974.40 ms /   153 runs   (   52.12 ms per token,    19.19 tokens per second)
llama_print_timings:       total time =    8886.56 ms /   161 tokens


213
[0.7681029796600342, 0.8954380035400391, 0.7977747917175293, 0.5795556545257569, 0.8273495674133301, 0.7513070106506348, 0.8209623336791992, 0.8299297332763672, 0.9034114837646484, 0.8177706718444824, 0.625499677658081, 0.877796745300293, 0.8191460609436035, 0.9320152282714844, 0.9164140701293946, 0.629492998123169, 0.8352705001831054, 0.6647021293640136, 0.7478991508483886, 0.8706809043884277, 0.7389620780944824, 0.8504174232482911, 0.6694564342498779, 0.796288251876831, 0.7814426898956299, 0.6269454002380371, 0.6292051792144775, 0.8863888740539551, 0.698784065246582, 0.8437670707702637, 0.7068845748901367, 0.8926518440246582, 0.7102718353271484, 0.5563716888427734, 0.7952279567718505, 0.8955549240112305, 0.8537315368652344, 0.8929434776306152, 0.7905792236328125, 0.8169466972351074, 0.486126708984375, 0.9188735961914063, 0.7866578102111816, 0.8280077934265136, 0.8444852828979492, 0.787843656539917, 0.7060833930969238, 0.7956328868865967, 0.8781953811645508, 0.7139519214630127, 0.